In [32]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: Qt5Agg


In [33]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [34]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [35]:
from problems.full_problem_max_latency import FullMaxLatencyProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 0.5e9
#requirements.max_throughput = 5e9
requirements.max_latency = 5400

problem = FullMaxLatencyProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 1000, 'n_offsprings': 50, 'n_max_gen': 10000}

settings = {}

# NSGA-II
from pymoo.algorithms.nsga2 import NSGA2
settings['NSGA-II'] = {
    'label': 'nsga2',
    'algorithm': NSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

# NSGA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization

ref_dirs = get_reference_directions("das-dennis", 4, n_partitions=13)
algo_settings['pop_size'] = int(np.ceil(len(ref_dirs) / 100) * 100)

print("# dirs: %d" % len(ref_dirs))

#get_visualization("scatter").add(ref_dirs).show()
settings['NSGA-III'] = {
    'label': 'nsga3',
    'algorithm': NSGA3(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=algo_settings['n_max_gen'],
                                                      n_max_evals=None)
}

# R-NSGA-II
from pymoo.algorithms.rnsga2 import RNSGA2
ref_points = np.array([
    [-1800e9, 45e3, 2000],   # min -throughput, min energy, d.c. pointing
    [-250e9, 60e3, 100]     # min -throughput, d.c. energy, min pointing
])
settings['RNSGA-II'] = {
    'label': 'rnsga2',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.001,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

# R-NSGA-II wide
from pymoo.algorithms.rnsga2 import RNSGA2
ref_points = np.array([
    [-1800e9, 45e3, 2000],   # min -throughput, min energy, d.c. pointing
    [-250e9, 60e3, 100]     # min -throughput, d.c. energy, min pointing
])
settings['RNSGA-II-e0_01'] = {
    'label': 'rnsga2_e0_01',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.01,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

setting = settings['NSGA-II']
setting = settings['NSGA-III']
# setting = settings['RNSGA-II']
# setting = settings['RNSGA-II-e0_01']

# dirs: 560


In [36]:
termination = setting['termination']
algorithm = setting['algorithm']

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               #save_history=True,
               verbose=True
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     600 |  0.00000E+00 |  2.70596E+07 |      21 |            - |            -
    2 |     650 |  0.00000E+00 |  3.59878E+06 |      22 |  0.010137271 |            f
    3 |     700 |  0.00000E+00 |  3.36513E+02 |      23 |  0.352766621 |        ideal
    4 |     750 |  0.00000E+00 |  2.06692E+02 |      21 |  0.051456913 |        nadir
    5 |     800 |  0.00000E+00 |  1.81253E+02 |      24 |  0.023035096 |        ideal
    6 |     850 |  0.00000E+00 |  1.62977E+02 |      26 |  0.422220789 |        nadir
    7 |     900 |  0.00000E+00 |  1.47273E+02 |      25 |  0.555539408 |        ideal
    8 |     950 |  0.00000E+00 |  1.32460E+02 |      26 |  0.010536861 |            f
    9 |    1000 |  0.00000E+00 |  1.20683E+02 |      30 |  0.030827245 |            f
   10 |    1050 |  0.00000E+00 |  1.11333E+02 |      34 |  0.012661025 |        ideal
   11 |    1100 |  0.00000E+00 |  1.01615E+02 |      3

In [37]:
pickle.dump((problem, setting, res), open('full_max_L_res_%s_%d_%d.pkl' % (setting['label'], algo_settings['pop_size'], algo_settings['n_max_gen']), 'wb'))


